# Retrieve data

In [36]:
import pandas as pd

# Load the dataframes from the CSV files
df_combined_vs = pd.read_csv('Combined_vs.csv', delimiter=';')
df_mcts_vs = pd.read_csv('MCTS_vs.csv', delimiter=';')
df_minimax_vs = pd.read_csv('MiniMax_vs.csv', delimiter=';')
df_logic_vs = pd.read_csv('Logic_vs.csv', delimiter=';')
df_random_vs = pd.read_csv('Random_vs.csv', delimiter=';')

# Merge the dataframes in a single one with only the common columns
common_cols = []
for col in df_combined_vs.columns:
    if col in df_mcts_vs.columns and col in df_minimax_vs.columns and col in df_logic_vs.columns and col in df_random_vs.columns:
        common_cols.append(col)
df = pd.concat(
    objs=[df_combined_vs[common_cols], df_mcts_vs[common_cols], df_minimax_vs[common_cols], df_logic_vs[common_cols], df_random_vs[common_cols]],
    ignore_index=True
)

df

,agent0,agent1,N_BATTLES,winrate,avg_n_turns,avg_n_switches,avg_hp_residue
0,Combined,Random,100,63.00,10.86,0.00,30.42
1,MCTS,Combined,100,52.00,10.48,1.00,23.49
2,MCTS,MiniMax,100,76.00,10.70,0.89,38.10
3,MCTS,MCTS,100,62.00,10.61,0.84,27.99
4,MCTS,Combined,300,56.33,10.37,0.91,26.19
5,MCTS,MCTS,300,58.00,10.53,0.99,25.86
6,MCTS,MiniMax,300,60.33,11.06,0.82,28.09
7,MCTS,Random,100,76.00,11.19,0.92,37.94
8,MCTS,Random,300,70.67,10.76,1.08,35.11
9,MCTS,Logic,100,71.00,11.71,4.15,38.98


# Winrate table

In [37]:

index_dict = {
    'Random': 0,
    'Logic': 1,
    'MiniMax': 2,
    'MCTS': 3,
    'Combined': 4
}
df_table_100 = pd.DataFrame(index=index_dict.keys(), columns=index_dict.keys())
df_table_300 = pd.DataFrame(index=index_dict.keys(), columns=index_dict.keys())


# Generate a dataframe with the winrate associated to all the combinations of battles
for agent in index_dict.keys():
    for opponent in index_dict.keys():
        if agent in df['agent0'] and opponent in df['agent1']:
            agent0 = agent
            agent1 = opponent
            complementary_winrate = False
            query = (df[agent0] == agent0) & (df[agent1] == agent1)
        else:
            agent0 = opponent
            agent1 = agent
            complementary_winrate = True
            query = (df['agent0'] == agent0) & (df['agent1'] == agent1)
        # Winrate for 100 battles
        row = df[query & (df['N_BATTLES'] == 100)]
        print(agent0, agent1)
        print(row)
        winrate = row['winrate'].values[0]
        df_table_100.loc[agent][opponent] = winrate if not complementary_winrate else 1 - winrate
        # Winrate for 300 battles
        row = df[query & (df['N_BATTLES'] == 300)]
        winrate = row['winrate'].values[0]
        df_table_300.loc[agent][opponent] = winrate if not complementary_winrate else 1 - winrate

df_table_100

Random Random
    agent0  agent1  N_BATTLES  winrate  avg_n_turns  avg_n_switches  \
21  Random  Random        100     58.0         12.7            1.59   

    avg_hp_residue  
21           22.06  
Logic Random
   agent0  agent1  N_BATTLES  winrate  avg_n_turns  avg_n_switches  \
20  Logic  Random        100     71.0         10.1            0.79   

    avg_hp_residue  
20           39.73  
MiniMax Random
     agent0  agent1  N_BATTLES  winrate  avg_n_turns  avg_n_switches  \
11  MiniMax  Random        100     47.0        14.13            0.64   

    avg_hp_residue  
11           21.38  
MCTS Random
  agent0  agent1  N_BATTLES  winrate  avg_n_turns  avg_n_switches  \
7   MCTS  Random        100     76.0        11.19            0.92   

   avg_hp_residue  
7           37.94  
Combined Random
     agent0  agent1  N_BATTLES  winrate  avg_n_turns  avg_n_switches  \
0  Combined  Random        100     63.0        10.86             0.0   

   avg_hp_residue  
0           30.42  


C:\Users\Gianluca\AppData\Local\Temp\ipykernel_372632\1550057984.py:30: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_table_100.loc[agent][opponent] = winrate if not complementary_winrate else 1 - winrate
C:\Users\Gianluca\AppData\Local\T

IndexError: index 0 is out of bounds for axis 0 with size 0

# Winrate diagram vs Random agent

In [19]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.bar(x1, y1, color='blue', alpha=0.7)
plt.title('Diagramma a barre 1')
plt.xlabel('Agents')
plt.ylabel('Winrate')
plt.show()

NameError: name 'x1' is not defined

<Figure size 1000x500 with 0 Axes>